In [120]:
import numpy as np
import pandas as pd
import re
import json
import math

In [130]:
df = pd.read_json('../collection/raw_data/offcampushousing.json')
col = ['address', 'price', 'bed', 'bath', 'area', 'company', 'neighborhood', 'laundry', 'pets', 'parking', 'utilities', 'property_type', 'year_built', 'description', 'images']
df[0:10]

,address,company,price,area,property_type,bed,bath,neighborhood,description,laundry,utilities,parking,pets,images
0,"514 South Forest Avenue Ann Arbor, MI 48104",Campus Realty,$965,335,Apartment,0,1,South University,Description\nAvailable for the 2020-2021 schoo...,0.0,"Gas, Heat, Recycling, Trash Removal, Water/Sew...",1.0,0.0,[//img.offcampusimages.com/BZpa6l-JfxacuCXaVmH...
1,"603 E Huron St Ann Arbor, MI 48104",American Campus Communities,"[$1,049 to $2,269, $1,439 to $1,539, $1,484 to...","[358, 358, 358, 358, 358, 358, 358]",Apartment,"[0, 2, 2, 3, 3, 4, 4]","[1.0, 1.0, 2.0, 3.0, 3.0, 2.0, 4.0]",Old Fourth Ward,Description\nHUBAnn Arbor\nStandard Apartment ...,1.0,"Cable, Gas, High-Speed Internet, Trash Removal...",1.0,0.0,[//img.offcampusimages.com/iGdgyCzJRtmlZa1UMc1...
2,"1300 S University Ave Ann Arbor, MI 48104",GreyStar,"[$1,149 to $2,249, $1,399 to $1,499, $1,149 to...","[450, 450, 450, 450, 450, 450, 450, 450, 450, ...",Apartment,"[0, 2, 4, 4, 4, 4, 5, 5, 6, 6, 3, 6]","[1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 3.0, 4.0, 4.0, ...",South University,Description\nLive at Landmark – the best in An...,1.0,"Cable, High-Speed Internet, Recycling, Trash R...",1.0,0.0,[//img.offcampusimages.com/diilurQlAhNu6V-Urju...
3,"401 Thompson Street Ann Arbor, MI 48104",Landmark Properties Management,"[$2,200, $1,570, $1,175]","[None, None, None]",Apartment,"[1, 2, 4]","[1.0, 2.0, 3.0]",Germantown,Description\nANN ARBOR STUDENT APARTMENTSJoin ...,1.0,"Cable, High-Speed Internet included",1.0,0.0,[//img.offcampusimages.com/pFZRsM7DK9Sv8Nku7pT...
4,"1701-1753 Broadview Lane Ann Arbor, MI 48105",Ann Arbor Apartments /CMB,"[$1,200 to $1,400, $1,550 to $1,700, $1,600 to...","[750 - 780, 750 - 780, 750 - 780]",Apartment,"[1, 2, 2]","[1.0, 1.5, 2.0]",West Murfin,Description\nLooking for a great apartment wit...,1.0,"Recycling, Trash Removal, Water/Sewer included",1.0,1.0,[//img.offcampusimages.com/ISKS99ttuaqE4spl-Pp...
5,"545 Landings Boulevard Ann Arbor, MI 48103",The Solomon Organization,"[$1,215 to $1,395, $1,365 to $1,770, $1,610 to...","[817-972, 817-972, 817-972]",Apartment,"[1, 2, 3]","[1.0, 2.0, 2.0]",Old West Side,Description\nWelcome home to Arbor Landings Ap...,1.0,No utilities included in rent,1.0,1.0,[//img.offcampusimages.com/aJyWJZ48DDhVScuemvC...
6,"420 Kellogg Street Ann Arbor, MI 48105",Friedman Intergrated Real Estate Solutions,"[$1,230 to $1,235, $1,460 to $1,480, $1,480 to...","[650, 650, 650]",Apartment,"[1, 2, 2]","[1.0, 1.0, 1.0]",North Campus,Description\nWelcome Home to Shoreview Apartme...,1.0,No utilities included in rent,1.0,1.0,[//img.offcampusimages.com/VTrcbZi8IFrSsFEK74c...
7,"220 N. First Ann Arbor, MI 48104",Ann Arbor Apartments /CMB,"[$1,350 to $1,400, $1,495 to $1,595]","[520, 520]",Apartment,"[0, 1]","[1.0, 1.0]",Old Fourth Ward,Description\nThese splendid studio and 1 bedro...,1.0,"Recycling, Trash Removal, Water/Sewer included",1.0,1.0,[//img.offcampusimages.com/nsv-KZv55SEIfPFxUbP...
8,"801 Miller Ann Arbor, MI 48103",Ann Arbor Apartments /CMB,"$1,400 to $1,600",700,Apartment,2,1,Old West Side,Description\nGreat location on scenic West Par...,1.0,"Gas, Heat, Water/Sewer included",1.0,1.0,[//img.offcampusimages.com/6mXfnnWkDJfgQxQAeck...
9,"712 W. Huron Ann Arbor, MI 48103",Ann Arbor Apartments /CMB,"[$1,400 to $1,575, $1,600 to $1,725]","[715, 715]",Apartment,"[1, 2]","[1.0, 1.0]",Old West Side,Description\n712 West Huron is a wonderful loc...,1.0,Water/Sewer included,1.0,1.0,[//img.offcampusimages.com/V0LfrKmaHKlsJm1GUNX...


In [131]:
final_rows = []
for row in df.itertuples():
    address = row[1]
    company = row[2]
    if row[5] != None:
        property_type = row[5].lower()
    else:
        property_type = row[5]
    neighborhood = row[8]
    if row[9] != None:
        description = row[9][12:]
    laundry = row[10]
    utilities = row[11]
    parking = row[12]
    pets = row[13]
    images = row[14]
    
    for j, s in enumerate(row[3]):
        
        if not isinstance(row[3], list):
            price = row[3]
        else:
            price = row[3][j]
            
        if not isinstance(row[6], list):
            bed = row[6]
        else:
            bed = row[6][j] 
            
        if not isinstance(row[7], list):
            bath = row[7]
        else:
            bath = row[7][j] 
            
        if not isinstance(row[4], list):
            area = row[4]
        else:
            area = row[4][j]   
            
        if math.isnan(pets):
            pets = int(0)

        entry = {'address': address,
                 'company': company, 
                 'property_type': property_type,
                 'neighborhood': neighborhood,
                 'price': price,
                 'bed': bed,
                 'bath': bath,
                 'area': area,
                 'description': description,
                 'laundry': int(laundry),
                 'utilities': utilities.split(', '),
                 'parking': int(parking),
                 'pets': pets,
                 'images': images,
                 'year_built': np.nan
                }
        final_rows.append(entry)
        
final_rows = pd.DataFrame(final_rows)

In [132]:
def clean_price(price):
    price = price.replace(',', '').replace('Person', '').replace('$', '').replace('/', '').strip()
    if len(price) < 7:
        return float(price)
    elif re.search('[Cc]all for [Rr]ent', price) != None:
        return np.nan
    elif re.search('-', price) != None:
        # mean recorded if price range given
        var = lambda x: (float(x[:x.find('-')].replace(',', '')) + float(x[x.find('-') + 1:].replace(',', ''))) / 2 
        return int(var(price))
    elif re.search('to', price) != None:
        # mean recorded if price range given
        var = lambda x: (float(x[:x.find('to')].replace(',', '')) + float(x[x.find('to') + 2:].replace(',', ''))) / 2 
        return float(var(price))

final_rows['price'] = final_rows['price'].apply(clean_price)

In [136]:
final_rows = final_rows[col]
final_rows['pets'] = final_rows['pets'].astype(int)

In [137]:
final_rows

,address,price,bed,bath,area,company,neighborhood,laundry,pets,parking,utilities,property_type,year_built,description,images
0,"514 South Forest Avenue Ann Arbor, MI 48104",965.0,0,1.0,335,Campus Realty,South University,0,0,1,"[Gas, Heat, Recycling, Trash Removal, Water/Se...",apartment,NaN,Available for the 2020-2021 school year\nThis ...,[//img.offcampusimages.com/BZpa6l-JfxacuCXaVmH...
1,"514 South Forest Avenue Ann Arbor, MI 48104",965.0,0,1.0,335,Campus Realty,South University,0,0,1,"[Gas, Heat, Recycling, Trash Removal, Water/Se...",apartment,NaN,Available for the 2020-2021 school year\nThis ...,[//img.offcampusimages.com/BZpa6l-JfxacuCXaVmH...
2,"514 South Forest Avenue Ann Arbor, MI 48104",965.0,0,1.0,335,Campus Realty,South University,0,0,1,"[Gas, Heat, Recycling, Trash Removal, Water/Se...",apartment,NaN,Available for the 2020-2021 school year\nThis ...,[//img.offcampusimages.com/BZpa6l-JfxacuCXaVmH...
3,"514 South Forest Avenue Ann Arbor, MI 48104",965.0,0,1.0,335,Campus Realty,South University,0,0,1,"[Gas, Heat, Recycling, Trash Removal, Water/Se...",apartment,NaN,Available for the 2020-2021 school year\nThis ...,[//img.offcampusimages.com/BZpa6l-JfxacuCXaVmH...
4,"603 E Huron St Ann Arbor, MI 48104",1659.0,0,1.0,358,American Campus Communities,Old Fourth Ward,1,0,1,"[Cable, Gas, High-Speed Internet, Trash Remova...",apartment,NaN,HUBAnn Arbor\nStandard Apartment Features:\nFu...,[//img.offcampusimages.com/iGdgyCzJRtmlZa1UMc1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2518,"1500 Gilbert Court Ann Arbor, MI 48105",879.5,10,10.0,130,None,North Campus,1,0,1,"[Electricity, Gas, Heat, High-Speed Internet, ...",room,NaN,Escher House is a non-profit cooperative house...,[//img.offcampusimages.com/BNBWVN7i01u9W4Zu0Qw...
2519,"1500 Gilbert Court Ann Arbor, MI 48105",879.5,10,10.0,130,None,North Campus,1,0,1,"[Electricity, Gas, Heat, High-Speed Internet, ...",room,NaN,Escher House is a non-profit cooperative house...,[//img.offcampusimages.com/BNBWVN7i01u9W4Zu0Qw...
2520,"1500 Gilbert Court Ann Arbor, MI 48105",879.5,10,10.0,130,None,North Campus,1,0,1,"[Electricity, Gas, Heat, High-Speed Internet, ...",room,NaN,Escher House is a non-profit cooperative house...,[//img.offcampusimages.com/BNBWVN7i01u9W4Zu0Qw...
2521,"1500 Gilbert Court Ann Arbor, MI 48105",879.5,10,10.0,130,None,North Campus,1,0,1,"[Electricity, Gas, Heat, High-Speed Internet, ...",room,NaN,Escher House is a non-profit cooperative house...,[//img.offcampusimages.com/BNBWVN7i01u9W4Zu0Qw...


In [138]:
final_rows.to_json(r'../data/offcampushousing.json')